In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.CIFAR10(root = './data',train = True, download = True,transform = transform)
testset = torchvision.datasets.CIFAR10(root = './data',train = False, download = True,transform = transform)

trainloader = torch.utils.data.DataLoader(trainset,batch_size = 64, shuffle=True, num_workers = 2)
testloader = torch.utils.data.DataLoader(testset,batch_size = 64, shuffle=True, num_workers = 2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from torch.nn.modules.pooling import MaxPool2d
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
       nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3,stride=1,padding=1),  # (i-f+2p)/s  + 1  (32-3+2)/1  +1
       nn.MaxPool2d(kernel_size=2,stride=2), # (M-P)/s + 1  (32-2)/2 + 1 = 16
       nn.ReLU(),
       nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3,stride=1,padding=1), 
       nn.MaxPool2d(kernel_size=2,stride=2),   # (M-P)/s + 1  (16-2)/2 +1  = 8
       nn.ReLU()
    )
    self.fc = nn.Sequential(
       nn.Linear(in_features=32*8*8,out_features=64),
       nn.ReLU(),
       nn.Linear(in_features=64,out_features=10)  
    )

  def forward(self,x):
    x = self.conv(x)
    x = x.view(x.size(0),-1)
    x = self.fc(x)
    return x

model = CNN().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01)

In [7]:
import time
start_time = time.time()

loss_list = []
accuracy_list = []
for epoch in range(30):
    epoch_loss = 0
    for images, labels in trainloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    loss_list.append(epoch_loss / len(trainloader))

        
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            accuracy = correct / total
            accuracy_list.append(accuracy)

        print("Epoch [{}/30] Training Accuracy: {:.4f}".format(epoch + 1, correct / total),"Loss: {:.4f}".format(loss_list[-1]))

print("Training took {:.2f} seconds".format(time.time() - start_time))

Epoch [1/30] Training Accuracy: 0.3449 Loss: 2.0977
Epoch [2/30] Training Accuracy: 0.4102 Loss: 1.7341
Epoch [3/30] Training Accuracy: 0.4610 Loss: 1.5525
Epoch [4/30] Training Accuracy: 0.4907 Loss: 1.4220
Epoch [5/30] Training Accuracy: 0.5242 Loss: 1.3283
Epoch [6/30] Training Accuracy: 0.4906 Loss: 1.2600
Epoch [7/30] Training Accuracy: 0.5662 Loss: 1.2049
Epoch [8/30] Training Accuracy: 0.5977 Loss: 1.1558
Epoch [9/30] Training Accuracy: 0.5066 Loss: 1.1157
Epoch [10/30] Training Accuracy: 0.6188 Loss: 1.0757
Epoch [11/30] Training Accuracy: 0.6269 Loss: 1.0381
Epoch [12/30] Training Accuracy: 0.5764 Loss: 1.0048
Epoch [13/30] Training Accuracy: 0.6555 Loss: 0.9750
Epoch [14/30] Training Accuracy: 0.6653 Loss: 0.9449
Epoch [15/30] Training Accuracy: 0.6911 Loss: 0.9156
Epoch [16/30] Training Accuracy: 0.6448 Loss: 0.8880
Epoch [17/30] Training Accuracy: 0.6486 Loss: 0.8653
Epoch [18/30] Training Accuracy: 0.6857 Loss: 0.8416
Epoch [19/30] Training Accuracy: 0.7162 Loss: 0.8209
Ep

In [8]:
with torch.no_grad():
  correct = 0 
  total = 0
  for images, labels in testloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy: {:.4f}'.format(correct/total))

Test Accuracy: 0.6533


In [9]:
save_state = {'model':model.state_dict(), 'optimizer':optimizer.state_dict()}
torch.save(save_state, 'CNN_GPU.pth')

In [8]:
state = torch.load('CNN_GPU.pth')
model.load_state_dict(state['model'])
optimizer.load_state_dict(state['optimizer'])

In [9]:
with torch.no_grad():
  correct = 0 
  total = 0
  for images, labels in testloader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy: {:.4f}'.format(correct/total))

Test Accuracy: 0.6533
